# Rolling Planning mit Flowtime Verfahren

In [2]:
import json
import pandas as pd
pd.set_option('display.max_rows', 20)

# Basics
import utils.basics.presenter as show
import utils.basics.converter as convert

import utils.checker as check
from ProductionDaySimulation import ProductionDaySimulation

In [3]:
import utils.rolling_planning.init_jobs_times as rp_init
import utils.rolling_planning.procedure as rp_proced

In [4]:
import utils.schedule_solver__arrival as ssv_a

## Generierung des Datensatzes für 6 Tage (Ankunft)

In [6]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
df_instance = convert.jssp_dict_to_df(instance)

df_jssp, df_times = rp_init.init_jobs_with_arrivals(df_instance, 7,  u_b_mmax = 0.9)
df_jssp

,Job,Operation,Machine,Processing Time
0,Job_000,0,M0,29
1,Job_000,1,M1,78
2,Job_000,2,M2,9
3,Job_000,3,M3,36
4,Job_000,4,M4,49
...,...,...,...,...
1325,Job_132,5,M9,76
1326,Job_132,6,M5,47
1327,Job_132,7,M3,52
1328,Job_132,8,M4,90


In [7]:
df_times

,Job,Arrival
0,Job_000,0.00
1,Job_001,88.05
2,Job_002,152.78
3,Job_003,207.97
4,Job_004,246.61
...,...,...
128,Job_128,9830.89
129,Job_129,9837.28
130,Job_130,9838.72
131,Job_131,9964.04


In [8]:
solver_limit = 60*5
solver_limit

300

## Params

In [10]:
solver_limit = 60*60*3  # 3 h
solver_limit

10800

In [11]:
# fix
day_length = 1440
horizon_days = 3

file_template = "data/06_schedule_{day:02d}.csv"

In [12]:
day_numb = 0
planning_end = 0

In [13]:
def day_number_to_prefix(n: int) -> str:
    s = ''
    while n >= 0:
        s = chr(n % 26 + 65) + s
        n = n // 26 - 1
    return s


# Initialisierung (Tag 0)

In [15]:
day_numb = 0

In [16]:
day_start = day_length*day_numb                      # 0
day_end = day_start + day_length                     # 1440
planning_end =  day_start + horizon_days*day_length  # 4320

In [17]:
# I)
df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_curr

,Job,Operation,Machine,Processing Time
0,Job_000,0,M0,29
1,Job_000,1,M1,78
2,Job_000,2,M2,9
3,Job_000,3,M3,36
4,Job_000,4,M4,49
...,...,...,...,...
565,Job_056,5,M9,76
566,Job_056,6,M5,47
567,Job_056,7,M3,52
568,Job_056,8,M4,90


In [18]:
df_times_curr 

,Job,Arrival
0,Job_000,0.00
1,Job_001,88.05
2,Job_002,152.78
3,Job_003,207.97
4,Job_004,246.61
...,...,...
52,Job_052,4070.89
53,Job_053,4077.28
54,Job_054,4078.72
55,Job_055,4204.04


### zusätzliche Jobs

In [20]:
extra_job_prefix = day_number_to_prefix(day_numb)
extra_job_prefix

'A'

In [21]:
"""
df_jssp_curr, df_times_curr = rp_init.add_extra_jobs_to_current_horizon(
    df_existing_jobs=df_jssp_curr,
    df_existing_arrivals=df_times_curr,
    df_template=df_jssp,
    start_time=day_start - 120,
    start_index=day_numb*10,
    job_prefix=extra_job_prefix,
    mean_interarrival_time=120,
    job_numb=1,
    shuffle=True
)
df_jssp_curr

"""

'\ndf_jssp_curr, df_times_curr = rp_init.add_extra_jobs_to_current_horizon(\n    df_existing_jobs=df_jssp_curr,\n    df_existing_arrivals=df_times_curr,\n    df_template=df_jssp,\n    start_time=day_start - 120,\n    start_index=day_numb*10,\n    job_prefix=extra_job_prefix,\n    mean_interarrival_time=120,\n    job_numb=1,\n    shuffle=True\n)\ndf_jssp_curr\n\n'

In [22]:
df_times_curr

,Job,Arrival
0,Job_000,0.00
1,Job_001,88.05
2,Job_002,152.78
3,Job_003,207.97
4,Job_004,246.61
...,...,...
52,Job_052,4070.89
53,Job_053,4077.28
54,Job_054,4078.72
55,Job_055,4204.04


### Jobs vorziehen

In [24]:
# Test
df_jssp_beforehand, df_times_beforehand = rp_init.sample_jobs_with_times_after_arrivaltime(df_jssp, df_times, 
                                                                             min_arrival_time=planning_end,
                                                                             n=2, random_state=123)
df_times_beforehand

,Job,Arrival
110,Job_110,8397.28
121,Job_121,9155.63


In [25]:
df_jssp_curr, df_times_curr = rp_init.add_beforehand_jobs_to_current_horizon(
    df_existing_jobs = df_jssp_curr,
    df_existing_times = df_times_curr,
    df_jssp = df_jssp,
    df_times = df_times,
    min_arrival_time = planning_end,
    n=3,
    random_state=23
)
df_times_curr

,Job,Arrival
0,Job_000,0.00
1,Job_001,88.05
2,Job_002,152.78
3,Job_003,207.97
4,Job_004,246.61
...,...,...
55,Job_055,4204.04
56,Job_056,4309.61
57,Job_074,5644.04
58,Job_095,7200.00


### Scheduling

In [ ]:
df_plan = ssv_a.solve_jssp_weighted_individual_flowtime(df_jssp_curr, df_times_curr , solver_time_limit=solver_limit, threads = 8)
df_plan

In [ ]:
df_plan.to_csv(file_template.format(day=day_numb), index = False)

In [ ]:
show.plot_gantt_machines(df_plan)
check.check_all_constraints(df_plan)

In [ ]:
show.plot_gantt_jobs(df_plan)

### Simulation

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)